# 20. 高级模块话题（Advanced Modules）

工程中常见模块话题：可选依赖、插件式导入、循环导入排查、reload 的局限。本节以“能用到”为主，不追求实现细节。

> 约定：Python 3.8；示例尽量只用标准库；代码块可直接运行。


## 前置知识

- 第 17-19 节：模块与包


## 知识点地图

- 1. 可选依赖：try/except ImportError
- 2. 动态导入：importlib.import_module
- 3. 循环导入：症状与处理思路
- 4. reload 局限：from x import y 不会自动更新 y


## 自检清单（学完打勾）

- [ ] 会用 try/except ImportError 处理可选依赖
- [ ] 了解 importlib.import_module 动态导入
- [ ] 理解循环导入的常见成因与基本排查思路
- [ ] 理解 reload 的局限（from import 不自动更新）


## 知识点 1：可选依赖：try/except ImportError

当第三方库可能不存在时，用 ImportError 回退到标准库或降级功能。


In [ ]:
try:
    import ujson as json
except ImportError:
    import json

print(json.dumps({'ok': True}))


## 知识点 2：动态导入：importlib.import_module

插件系统常用“字符串模块名”动态导入，并捕获异常。


In [ ]:
import importlib

name = 'math'
mod = importlib.import_module(name)
print(mod.sqrt(9))


## 知识点 3：循环导入：症状与处理思路

循环导入常见于：
- 两个模块互相 import
- 顶层 import 触发执行顺序问题

处理思路：
- 把 import 移到函数内部（延迟导入）
- 抽公共代码到第三个模块
- 避免顶层执行依赖另一个模块初始化的代码



## 知识点 4：reload 局限：from x import y 不会自动更新 y

reload 会更新模块对象，但已导入到当前命名空间的名字不会自动重绑定。


## 常见坑

- 不要在 __init__.py 导入大量重量级模块（会增加启动成本/引入循环）
- 动态导入要做异常处理与安全校验（避免导入不可信模块名）


## 综合小案例：实现一个简单插件加载器（字符串模块名列表）

给定模块名列表 plugins，逐个 import，成功则记录到 dict，失败则记录错误信息。


In [ ]:
import importlib

def load_plugins(names):
    loaded = {}
    errors = {}
    for name in names:
        try:
            loaded[name] = importlib.import_module(name)
        except Exception as e:
            errors[name] = f"{type(e).__name__}: {e}"
    return loaded, errors

mods, errs = load_plugins(['math', 'this_module_does_not_exist'])
print('loaded:', list(mods))
print('errors:', errs)


## 自测题（不写代码也能回答）

- 可选依赖如何写？
- 动态导入常用哪个标准库函数？
- 循环导入常见的解决方法有哪些？


## 练习题（建议写代码）

- 把插件加载器改成：只允许导入以 demo_ 开头的模块名。
- 写一个例子演示：from math import sqrt 后，reload(math) 是否会改变 sqrt？解释原因。
